# Projeto Marinha do Brasil
### Autor: Vinícius dos Santos Mello (viniciusdsmello@poli.ufrj.br)
### Laboratorio de Processamento de Sinais - UFRJ

In [ ]:
import os
import pickle
import numpy as np
import time

from sklearn.decomposition import PCA
from sklearn.externals import joblib

from Functions import ClassificationAnalysis as class_anal

init_time = time.time()

m_time = time.time()
print 'Time to import all libraries: '+str(m_time-init_time)+' seconds'

analysis_name = 'NeuralNetwork'
data_path = os.getenv('OUTPUTDATAPATH')
results_path = os.getenv('PACKAGE_NAME')

pict_results_path = results_path+'/'+analysis_name+'/picts'
files_results_path = results_path+'/'+analysis_name+'/output_files'

# Read data
# Check if LofarData has created...
m_time = time.time()

database = '4classes'
n_pts_fft = 1024
decimation_rate = 3
spectrum_bins_left = 400
development_flag = False
development_events = 100


if not os.path.exists('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                      (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left)):
    print 'No Files in %s/%s\n'%(data_path,database)
else:
    #Read lofar data
    [data,trgt,class_labels] = joblib.load('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                                           (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left))


    m_time = time.time()-m_time
    print 'Time to read data file: '+str(m_time)+' seconds'

    # correct format
    all_data = data
    all_trgt = trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))
    
    # Process data
    # unbalanced data to balanced data with random data creation of small classes

    # Same number of events in each class
    qtd_events_biggest_class = 0
    biggest_class_label = ''

    for iclass, class_label in enumerate(class_labels):
        if sum(all_trgt==iclass) > qtd_events_biggest_class:
            qtd_events_biggest_class = sum(all_trgt==iclass)
            biggest_class_label = class_label
        print "Qtd event of %s is %i"%(class_label,sum(all_trgt==iclass))
    print "\nBiggest class is %s with %i events"%(biggest_class_label,qtd_events_biggest_class)


    balanced_data = {}
    balanced_trgt = {}

    from Functions import DataHandler as dh
    m_datahandler = dh.DataHandlerFunctions()

    for iclass, class_label in enumerate(class_labels):
        if development_flag:
            class_events = all_data[all_trgt==iclass,:]
            if len(balanced_data) == 0:
                balanced_data = class_events[0:development_events,:]
                balanced_trgt = (iclass)*np.ones(development_events)
            else:
                balanced_data = np.append(balanced_data,
                                          class_events[0:development_events,:], 
                                          axis=0)
                balanced_trgt = np.append(balanced_trgt,(iclass)*np.ones(development_events))
        else:
            if len(balanced_data) == 0:
                class_events = all_data[all_trgt==iclass,:]
                balanced_data = m_datahandler.CreateEventsForClass(
                    class_events,qtd_events_biggest_class-(len(class_events)))
                balanced_trgt = (iclass)*np.ones(qtd_events_biggest_class)
            else:
                class_events = all_data[all_trgt==iclass,:]
                created_events = (m_datahandler.CreateEventsForClass(all_data[all_trgt==iclass,:],
                                                                     qtd_events_biggest_class-
                                                                     (len(class_events))))
                balanced_data = np.append(balanced_data,created_events,axis=0)
                balanced_trgt = np.append(balanced_trgt,
                                          (iclass)*np.ones(created_events.shape[0]),axis=0)
        
    all_data = balanced_data
    all_trgt = balanced_trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))

# Train process
## The train will modify one file and create three different files

### Log File:
This file will store basic information of all Package's trains and it will guide the analyses file to recognize which train information file should load. In each train this file should be appended with a new line contend the basic information to find the train information file (TXT FORMAT) or (PYTHON FORMAT) - This file should be access by all programs (MatLab and Python) for Analysis

### Train Information File
This file will store full information of the train performed (all parameters) in its name (each train information file will have a different name). And it will guide which train classifier file or which train result file should be open for analysis (TXT FORMAT) or (PYTHON FORMAT) - This file should be access by all programs (MatLab and Python) for Analysis

### Train Result File
This file will store the classifier result for all data and classification target (TXT FORMAT) or (PYTHON FORMAT) - This file should be access by all programs (MatLab and Python) for Analysis


In [ ]:
%time

from sklearn import cross_validation
from Functions import LogFunctions as log

# Create a entry in log file
m_log = log.LogInformation()
date = m_log.CreateLogEntry("Classification",'NeuralNetwork')

# Create a train information file
n_folds = 2
n_inits = 2
norm = 'mapstd'

train_info = {}
train_info['n_folds'] = n_folds
train_info['n_inits'] = n_inits
train_info['norm'] = norm

# divide data in train and test for novelty detection
print 'Dividing data in train and test'
CVO = cross_validation.StratifiedKFold(all_trgt, n_folds)
CVO = list(CVO)
train_info['CVO'] = CVO

train_info['preprocessing_extraction_done'] = False
train_info['preprocessing_analysis_done'] = False
train_info['train_done'] = False
train_info['results_done'] = False
train_info['dev'] = development_flag

train_info_name = files_results_path+'/'+date+'_train_info.jbl'
joblib.dump([train_info],train_info_name,compress=9)

In [ ]:
# Read log files
from Functions import LogFunctions as log
mlog = log.LogInformation()
log_entries = mlog.RecoverLogEntries(package_name="Classification")
lastTrain = log_entries[np.max(log_entries.keys())]
print 'Last Train\nDate: {0}\nPackage: {1}\n'.format(lastTrain['date'],lastTrain['package'])
print log_entries

In [ ]:
a = class_anal.NeuralClassification(name         = analysis_name,
                                    preproc_path = files_results_path,
                                    train_path   = files_results_path,
                                    anal_path    = files_results_path)
date = '2017_07_11_23_19_15'
n_epochs = 200
a.trn_info = class_anal.TrnInformation(date     = lastTrain['date'], #date,
                                       n_folds  = n_folds,
                                       verbose  = False,
                                       n_inits  = n_inits,
                                       n_epochs = n_epochs,
                                       batch_size = 512)
a.trn_info.SplitTrainSet(all_trgt)
a.trn_info.Print()

In [ ]:
%matplotlib inline  
for ifold in range(n_folds):
    a.analysis_train_plot(data = all_data,trgt = trgt_sparse,trn_info = a.trn_info, n_neurons= 25, fold = ifold)
#a.train(all_data,trgt_sparse,trn_info=a.trn_info, n_neurons=1, fold=0)

In [ ]:
%matplotlib inline  
for ifold in range(n_folds):
    a.analysis_conf_mat(data = all_data,
                        trgt = trgt_sparse,
                        trn_info = a.trn_info,
                        class_labels=class_labels.values(),
                        n_neurons=25,
                        fold=ifold)

In [ ]:
%matplotlib inline  
for ifold in range(n_folds):
    a.analysis_output_hist(data = all_data,trgt = trgt_sparse,trn_info = a.trn_info, n_neurons= 25, fold = ifold)

In [ ]:
%matplotlib inline  
a.analysis_top_sweep(all_data,trgt_sparse,trn_info=a.trn_info, min_neurons=15, max_neurons=25)